Error opening webcam
Error capturing frame


In [3]:
import os
import cv2
import numpy as np
from skimage import exposure

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

def trainPerceptron(train_set, train_labels, learning_rate, max_iter):
    input_size = train_set.shape[1]
    
    # Initialize weights and bias
    W = np.random.randn(input_size)
    b = 0
    
    for iteration in range(max_iter):
        for i, image in enumerate(train_set):
            # Forward propagation
            output = sigmoid(np.dot(image, W) + b)
            
            # Backpropagation
            output_error = train_labels[i] - output
            
            # Update weights and bias using gradient descent
            W += learning_rate * image * output_error
            b += learning_rate * output_error
    
    return W, b

def classifyPerceptron(W, b, dev_set):
    ret = []
    for image in dev_set:
        output = sigmoid(np.dot(image, W) + b)
        if output <= 0.5:
            ret.append(0)
        else:
            ret.append(1)
    return ret

C:\Users\norla\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [12]:
import numpy as np

num_classes = 4 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def grad_log(theta, X, y):
    m = X.shape[0]
    h = sigmoid(X @ theta)
    grad = (1 / m) * (X.T @ (h - y))
    return grad

def cost_log(theta, X, y):
    m = X.shape[0]
    h = sigmoid(X @ theta)
    cost = (-1 / (2*m)) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))
    return cost

def trainPerceptron(train_set, train_labels, learning_rate, max_iter):
    input_size = train_set.shape[1]
    theta = np.zeros((num_classes, input_size))

    for c in range(num_classes):
        y = (train_labels == c).astype(int)
        for _ in range(max_iter):
            grad = grad_log(theta[c], train_set, y)
            theta[c] -= learning_rate * grad

    return theta

def classifyPerceptron(W, X):
    probabilities = sigmoid(X @ W.T)
    return np.argmax(probabilities, axis=1)


train_set = hand_array  
train_labels = np.concatenate((2*np.ones(1000), np.ones(1880), np.zeros(470), 3*np.ones(572))) 

In [14]:
alpha = 10**(-6)
num_iter = 10000

# Add a column of ones to train_data
ones = np.ones((train_set.shape[0], 1))
train_X = np.hstack((ones, train_set))

# Train the perceptron
theta = trainPerceptron(train_X, train_labels, alpha, num_iter)

# Classify using the trained perceptron
predictions = classifyPerceptron(theta, train_X)

# Print predictions or use them as needed
print("Predictions:", predictions)

accuracy = [0, 0, 0, 0]

counter = 0
for i in predictions:
    if i == train_labels[counter]:
        accuracy[i] += 1
    counter += 1
accuracy[0] /= 470
accuracy[1] /= 1880
accuracy[2] /= 1000
accuracy[3] /= 572

Predictions: [2 2 2 ... 3 3 3]


In [15]:
print(accuracy)

[0.02127659574468085, 1.0, 1.0, 1.0]


In [9]:
from PIL import Image

hand_set = []
count = 0

files = os.listdir(r'C:\Users\norla\OneDrive\Desktop\ThumbsUpSet')
files2 = os.listdir(r'C:\Users\norla\OneDrive\Desktop\ThumbsDownSet')
files3 = os.listdir(r'C:\Users\norla\OneDrive\Desktop\archive\Hands\Hands')
files4 = os.listdir(r'C:\Users\norla\OneDrive\Desktop\FistSet')

for file_name in files3:
    if file_name.endswith('.jpg'):
        file_path = os.path.join(r'C:\Users\norla\OneDrive\Desktop\archive\Hands\Hands', file_name)
        image = Image.open(file_path)
        
        np_image = np.array(image)
        
        # Convert the image to grayscale
        gray = cv2.cvtColor(np_image, cv2.COLOR_RGB2GRAY)  

        # Threshold the image to separate the hand from the background
        _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

        # Find contours in the binary image
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Assuming the largest contour corresponds to the hand
        largest_contour = max(contours, key=cv2.contourArea)

        # Get the bounding rectangle of the largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Crop the hand region from the original image
        hand_crop = gray[y:y+h, x:x+w]

        # Resize the cropped hand region to 64x64
        hand_resized = cv2.resize(hand_crop, (64, 64))
        
        # Add image to set
        hand_set.append(hand_resized.flatten())
        
        count += 1
        if count >= 1000:
            break


from PIL import Image

# Load images from ThumbsUpSet directory
for i in range(20):
    for file_name in files:
        if file_name.endswith('.jpg'):
            file_path = os.path.join(r'C:\Users\norla\OneDrive\Desktop\ThumbsUpSet', file_name)
            image = Image.open(file_path)
            gray = image.convert('L')  # Convert to grayscale
            hand_resized = gray.resize((64, 64))
            hand_set.append(np.array(hand_resized).flatten())

# Load images from ThumbsDownSet directory
for i in range(5):
    for file_name in files2:
        if file_name.endswith('.jpg'):
            file_path = os.path.join(r'C:\Users\norla\OneDrive\Desktop\ThumbsDownSet', file_name)
            image = Image.open(file_path)
            gray = image.convert('L')  # Convert to grayscale
            hand_resized = gray.resize((64, 64))
            hand_set.append(np.array(hand_resized).flatten())
            
# Load images from FistSet directory
for i in range(4):
    for file_name in files4:
        if file_name.endswith('.jpg'):
            file_path = os.path.join(r'C:\Users\norla\OneDrive\Desktop\FistSet', file_name)
            image = Image.open(file_path)
            gray = image.convert('L')  # Convert to grayscale
            hand_resized = gray.resize((64, 64))
            hand_set.append(np.array(hand_resized).flatten())

In [10]:
hand_array = np.stack(hand_set)
print(hand_array.shape)

(3922, 4096)
